In [68]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import re
from datetime import datetime, timedelta
import os
import oracledb
from sqlalchemy import create_engine

title = 'data engineer'
location = "St Louis, Missouri, United States" 
location = "Chicago, Illinois, United States"
#location = "United States" 
job_count = 0
exp_level = "2"  # 2==entry level
post_date = "r604800"  # r604800==Past week

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}&start={job_count}'


print(url)

https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_E=2&f_TPR=r604800&keywords="data engineer"&location=Chicago, Illinois, United States&start=0


### FUNCTION DEFINITIONS

In [69]:
# Sends a request for one results page to LinkedIn to get a list of job titles
def get_jobs_page(job_count) -> list:
    url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}&start={job_count}'
    print("job page URL: ", url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    all_job_titles = soup.find_all("li")
    return all_job_titles

# Parses out the job title from one listing
def parse_job_id(job):
    return job.find("div", {"class":"base-card"}).get("data-entity-urn").split(":")[3]

# Get all job ids from one results page
def get_job_ids(job_count):
    job_ids = []
    all_job_titles = get_jobs_page(job_count)
    if not all_job_titles:  # if no jobs titles in the list
        print("No job titles found!")
        return None 

    all_job_ids = list(map(parse_job_id, all_job_titles))
    print("all job ids: ", all_job_ids)
    return all_job_ids
    



# I can't figure out how to get the count, the below function is useless
def get_result_count():

    base_url = "https://www.linkedin.com/jobs/search"
    url = f'{base_url}?f_E={exp_level}&f_TPR={post_date}&keywords="{title}"&location={location}'
    print(url)
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    print(soup)
    count = soup.find("div", {"class":"jobs-search-results-list__subtitle"})
    print(count)




In [70]:
job_ids = []
job_count = 0
max_jobs_to_scrape = 25
# I can't get a job_count for the search query, 
# so going to scrape till it can NOT find any more job ids
# or I'm going to scrape upto 300 jobs or 12 pages
while job_count < max_jobs_to_scrape:
    page_of_ids = get_job_ids(job_count)
    if page_of_ids:
        job_ids.extend(page_of_ids)
    else:
        break
    job_count += 25
    
print(job_ids)

job page URL:  https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_E=2&f_TPR=r604800&keywords="data engineer"&location=Chicago, Illinois, United States&start=0


all job ids:  ['3708984293', '3674224518', '3676035101', '3610297973', '3705364410', '3708958224', '3708954564', '3708953838']
['3708984293', '3674224518', '3676035101', '3610297973', '3705364410', '3708958224', '3708954564', '3708953838']


In [71]:
print(job_ids)


['3708984293', '3674224518', '3676035101', '3610297973', '3705364410', '3708958224', '3708954564', '3708953838']


In [72]:
job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
l_all_job_info = []

for id in job_ids:
    d_job_info = {}
    d_job_info["job_id"] = int(id)
    job_desc_url = job_url.format(id)
    print(job_desc_url)
    res = requests.get(job_desc_url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    # Get the company name
    d_job_info["company"] = soup.find("div", {"class":"top-card-layout__card"}).find("a").find("img").get("alt")

    # Get the location
    d_job_info["location"] = soup.find("div", {"class":"topcard__flavor-row"}).find("span", {"class":"topcard__flavor--bullet"}).text.strip()

    # Get the job title
    d_job_info["job_title"] = soup.find("h2", {"class":"top-card-layout__title"}).text.strip()

    # Get the full job description 
    d_job_info["job_description"] = soup.find("div", {"class":"show-more-less-html__markup"}).get_text(separator=u"\n")
    print(len(d_job_info["job_description"]))
    
    # Get years of experience!!!
    d_job_info["experience"] = re.findall(r".*\D\d{1,2}\D.*years?", d_job_info["job_description"])
    d_job_info["experience"] = "\n".join(d_job_info["experience"])

    # Get Seniority level, Employment type, Job function, Industries
    job_criteria_list = soup.find("ul", {"class":"description__job-criteria-list"}).find_all("li")
    for criteria in job_criteria_list:
        criteria = criteria.text.split("\n") # convert lines to a list
        criteria = [i.strip() for i in criteria if i.strip()] # remove lines with only white space
        criteria[0] = criteria[0].replace(" ", "_").lower()
        d_job_info.update({criteria[0]:criteria[1]})

    # Get job posting date
    posting_date = soup.find("span", {"class":"posted-time-ago__text"}).text.strip()
    posting_num = int(re.match(r'\d{1,2}',posting_date).group())
    if "minute" in posting_date:
        d_job_info["posting_date"] = datetime.today() - timedelta(minutes=posting_num)
    elif "hour" in posting_date:
        d_job_info["posting_date"] = datetime.today() - timedelta(hours=posting_num)
    elif "day" in posting_date:
        d_job_info["posting_date"] = datetime.today() - timedelta(days=posting_num)
    else:
        d_job_info["posting_date"] = ""
    
    
    # !!! Get Other useful info with AI !!!

    
    # Get URL 
    d_job_info["url"] = soup.find("a", {"class":"topcard__link"}).get("href")
    print(d_job_info["url"])

    # Append the job info (dict) to the list of job info
    l_all_job_info.append(d_job_info)

# Convert list of all job info dicts to a dataframe
df_all_job_info = pd.DataFrame(l_all_job_info)
df_all_job_info


https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3708984293
1024
https://www.linkedin.com/jobs/view/data-engineer-vizient-at-shiftcode-analytics-inc-3708984293?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3674224518
5699
https://www.linkedin.com/jobs/view/informatica-iics-data-engineer-at-old-national-bank-3674224518?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3676035101
5699
https://www.linkedin.com/jobs/view/informatica-iics-data-engineer-at-old-national-bank-3676035101?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3610297973
5197
https://www.linkedin.com/jobs/view/azure-data-engineer-administrator-at-medline-industries-lp-3610297973?trk=public_jobs_topcard-title
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3705364410
3571
https://www.linkedin.com/jobs/view/senior-associate-data-engineering-remote-wfh-at-get-it-recruit-information-technology-370

,job_id,company,location,job_title,job_description,experience,seniority_level,employment_type,job_function,industries,posting_date,url
0,3708984293,"ShiftCode Analytics, Inc.","Chicago, IL",Data Engineer : Vizient,\n\nHybrid 2-3 days onsite\nInterview process ...,,Entry level,Full-time,Information Technology,Information Technology & Services,2023-09-09 04:04:26.634148,https://www.linkedin.com/jobs/view/data-engine...
1,3674224518,Old National Bank,"Chicago, IL",Informatica IICS Data Engineer,\n\nOverview\nOld National Bank has been servi...,Over 5 years of experience with Informatica d...,Entry level,Full-time,Information Technology,Banking,2023-09-07 04:04:26.973700,https://www.linkedin.com/jobs/view/informatica...
2,3676035101,Old National Bank,"Tinley Park, IL",Informatica IICS Data Engineer,\n\nOverview\nOld National Bank has been servi...,Over 5 years of experience with Informatica d...,Entry level,Full-time,Information Technology,Banking,2023-09-08 04:04:27.295702,https://www.linkedin.com/jobs/view/informatica...
3,3610297973,"Medline Industries, LP","Northfield, IL",Azure Data Engineer/Administrator,\n Medline Industries continues to grow...,"Medline Industries continues to grow, ...",Entry level,Full-time,Information Technology,Medical Equipment Manufacturing,2023-09-05 04:04:27.704701,https://www.linkedin.com/jobs/view/azure-data-...
4,3705364410,Get It Recruit - Information Technology,"Chicago, IL",Senior Associate Data Engineering - Remote | WFH,\n We are a leading digital transformat...,,Entry level,Full-time,Engineering and Information Technology,Human Resources Services,2023-09-06 04:04:28.017703,https://www.linkedin.com/jobs/view/senior-asso...
5,3708958224,BDO USA,"Chicago, IL",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:04:28.352850,https://www.linkedin.com/jobs/view/tax-digital...
6,3708954564,BDO USA,"Rosemont, IL",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:04:28.770667,https://www.linkedin.com/jobs/view/tax-digital...
7,3708953838,BDO USA,"Oak Brook, IL",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:04:29.163202,https://www.linkedin.com/jobs/view/tax-digital...


### Get all the job IDs that already exist in the database

In [73]:
cs = '''(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.us-sanjose-1.oraclecloud.com))(connect_data=(service_name=ga3e236c6957ba6_oltpdb_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))'''
user="appuser"
password=os.environ['ORACLE_PASSWORD_APPUSER']
engine = create_engine(
    f'oracle+oracledb://{user}:{password}@{cs}'
    )

with engine.connect() as conn:
    df_jobs_exists = pd.read_sql_query(f'SELECT job_id FROM tbl_jobs WHERE job_id IN ({",".join(job_ids)})', conn)
               
df_jobs_exists

,job_id


### Filter out all the IDs that exist in the database

In [74]:
df_all_job_info = df_all_job_info[~df_all_job_info['job_id'].isin(list(df_jobs_exists['job_id']))]
df_all_job_info

,job_id,company,location,job_title,job_description,experience,seniority_level,employment_type,job_function,industries,posting_date,url
0,3708984293,"ShiftCode Analytics, Inc.","Chicago, IL",Data Engineer : Vizient,\n\nHybrid 2-3 days onsite\nInterview process ...,,Entry level,Full-time,Information Technology,Information Technology & Services,2023-09-09 04:04:26.634148,https://www.linkedin.com/jobs/view/data-engine...
1,3674224518,Old National Bank,"Chicago, IL",Informatica IICS Data Engineer,\n\nOverview\nOld National Bank has been servi...,Over 5 years of experience with Informatica d...,Entry level,Full-time,Information Technology,Banking,2023-09-07 04:04:26.973700,https://www.linkedin.com/jobs/view/informatica...
2,3676035101,Old National Bank,"Tinley Park, IL",Informatica IICS Data Engineer,\n\nOverview\nOld National Bank has been servi...,Over 5 years of experience with Informatica d...,Entry level,Full-time,Information Technology,Banking,2023-09-08 04:04:27.295702,https://www.linkedin.com/jobs/view/informatica...
3,3610297973,"Medline Industries, LP","Northfield, IL",Azure Data Engineer/Administrator,\n Medline Industries continues to grow...,"Medline Industries continues to grow, ...",Entry level,Full-time,Information Technology,Medical Equipment Manufacturing,2023-09-05 04:04:27.704701,https://www.linkedin.com/jobs/view/azure-data-...
4,3705364410,Get It Recruit - Information Technology,"Chicago, IL",Senior Associate Data Engineering - Remote | WFH,\n We are a leading digital transformat...,,Entry level,Full-time,Engineering and Information Technology,Human Resources Services,2023-09-06 04:04:28.017703,https://www.linkedin.com/jobs/view/senior-asso...
5,3708958224,BDO USA,"Chicago, IL",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:04:28.352850,https://www.linkedin.com/jobs/view/tax-digital...
6,3708954564,BDO USA,"Rosemont, IL",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:04:28.770667,https://www.linkedin.com/jobs/view/tax-digital...
7,3708953838,BDO USA,"Oak Brook, IL",Tax Digital Transformation & Innovation Senior...,\n\nJob Description\nJob Summary:\nThe Digital...,Bachelor's degree and six (6) or more years o...,Entry level,Full-time,Finance and Accounting/Auditing,Accounting,2023-09-09 04:04:29.163202,https://www.linkedin.com/jobs/view/tax-digital...


### Insert job info (dataframe) into the database

In [75]:
df_all_job_info.to_sql('tbl_jobs', engine, 'appuser', if_exists='append', index=False, method=None)

-1

### Double Check database insert

In [76]:
with engine.connect() as conn:
    df_check_database = pd.read_sql_query(f'SELECT job_id FROM tbl_jobs WHERE job_id IN ({",".join(job_ids)})', conn)

df_check_database

,job_id
0,3610297973
1,3674224518
2,3676035101
3,3705364410
4,3708953838
5,3708954564
6,3708958224
7,3708984293
